# Байесовский выбор субоптимальной структуры

In [ ]:
import torch as t
import torchvision
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pylab as plt
from torch.nn.utils import clip_grad_value_
%matplotlib inline
import pickle


In [ ]:
input_dim = 32*32*1 
class_num = 10
n_epochs =  20
fine_tune_epochs = 10
batch_size = 256
random_seed = 42
valid_size = 0.1 # валидация не используется. Сохраняем ее для чистоты эксперимента
trials = 10
search_space = [1, 16, 32, 64, 256, 512, 1024]  # '1' кодирует тождественное отображение

In [ ]:
train_data = torchvision.datasets.CIFAR10('./files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                  torchvision.transforms.Lambda(lambda x: x.mean(0).view(-1))
                             ]))

test_data = torchvision.datasets.CIFAR10('./files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                              (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                  torchvision.transforms.Lambda(lambda x: x.mean(0).view(-1))
                             ]))

num_train = len(train_data)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train))

train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = t.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler, num_workers=0, pin_memory=True )
test_loader = t.utils.data.DataLoader(test_data, batch_size=batch_size)
valid_loader = t.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler)


In [ ]:
init_sigma = -3.0
init_h_sigma = 0.0
class VarLayer(nn.Module):
    def __init__(self, in_, out_, device='cuda',  act=F.tanh):
        nn.Module.__init__(self)        
        self.mean = nn.Parameter(t.randn(in_, out_).cuda())
        t.nn.init.xavier_uniform(self.mean)
        self.log_sigma = nn.Parameter(t.ones(in_, out_).cuda()*(init_sigma))    
        self.mean_b = nn.Parameter(t.randn(out_).cuda())
        self.log_sigma_b = nn.Parameter(t.ones(out_).cuda()*(init_sigma))
        
        
        self.h_sigma = nn.Parameter(t.ones(in_, out_).cuda()*(init_h_sigma))    
        self.h_sigma_b = nn.Parameter(t.ones(out_).cuda()*(init_h_sigma))   
         
        self.act = act
        
    def forward(self,x, gamma=1.0):
        if self.training:
            self.eps_w = t.distributions.Normal(self.mean, t.exp(self.log_sigma))
            self.eps_b = t.distributions.Normal(self.mean_b, t.exp(self.log_sigma_b))
        
            w = self.eps_w.rsample()
            b = self.eps_b.rsample()
        else:            
            w = self.mean
            b = self.mean_b
        return self.act(t.matmul(x, w)+b)*gamma

    def KLD(self, weights = 1.0, weights_b = 1.0):
        self.eps_w = t.distributions.Normal(self.mean * weights, t.exp(self.log_sigma) *weights)
        self.eps_b = t.distributions.Normal(self.mean_b * weights_b, t.exp(self.log_sigma_b) * weights_b)
        self.h_w = t.distributions.Normal(t.zeros(self.mean.size()).cuda(), t.exp(self.h_sigma)*weights)
        self.h_b = t.distributions.Normal(t.zeros(self.mean.size()).cuda(), t.exp(self.h_sigma_b) * weights_b)
        
        
        k1 = t.distributions.kl_divergence(self.eps_w,self.h_w).sum()        
        k2 = t.distributions.kl_divergence(self.eps_b,self.h_b).sum()        
        return k1+k2

In [ ]:
class MixedLayer(nn.Module):
    def __init__(self, in_, dims, lambda_t, act=F.tanh):
        nn.Module.__init__(self)    
        self.lambda_t = lambda_t
        self.dims = dims
        self.layer = VarLayer(in_, max(dims), act=act)
        
        self.s = nn.Parameter(t.ones(len(dims), device='cuda')+ t.randn(len(dims), device='cuda')*.0)    
        self.s_prior = nn.Parameter(t.ones(len(dims), device='cuda')+ t.randn(len(dims), device='cuda')*.0)
        self.log_t = nn.Parameter(t.zeros(1, device='cuda'))           
        self.t = t.exp(self.log_t)+0.2 
        # мы добавляем шум в слои для уменьшения возможной подстройки одного слоя под другой
        # в противном случае становится более вероятным случай, когда
        # в первую очередь выбирается наиболее простая подмодель, а остальные "достраиваются"
        # с небольшими значениями для структурных параметров
        self.noises = [(t.randn(d).cuda()*1.0)  if d != 1 else t.randn(in_).cuda() for d in dims]
        
        
    def forward(self, x, noise=0.0):        
        self.t = t.exp(self.log_t)+0.2
        self.gamma = t.distributions.RelaxedOneHotCategorical(self.t, logits=self.s)        
        gamma = self.gamma.rsample()
        var_result = self.layer(x, 1.0)
        result = t.zeros(x.shape).cuda()
        for i,d in enumerate(self.dims):
            if d == 1:
                result = result + (x+self.noises[i])*gamma[i] 
            else:
                result[:,:d] =result[:,:d] +  (var_result[:,:d]+self.noises[i])*gamma[i]            
        return result
    
    def kld_normal(self): 
        self.t = t.exp(self.log_t)+0.2
        self.gamma = t.distributions.RelaxedOneHotCategorical(self.t, logits=self.s)        
        sample = (self.gamma.rsample()+0.0001)
        weights = t.zeros(self.layer.mean.size()).cuda()+0.0001
        weights_b = t.zeros(self.layer.mean_b.size()).cuda()+0.0001
        for i in range(len(self.dims)):
            weights[:,:self.dims[i]]+=sample[i]
            weights_b[:self.dims[i]]+=sample[i]
            
        
        return self.layer.KLD(weights, weights_b)
        
    
    
    
    def kld_structure(self):        
        self.t = t.exp(self.log_t)+0.2
        self.gamma_prior = t.distributions.RelaxedOneHotCategorical(self.lambda_t, logits=self.s_prior)
        self.gamma = t.distributions.RelaxedOneHotCategorical(self.t, logits=self.s)
        
        sample = (self.gamma.rsample()+0.0001)
        return self.gamma.log_prob(sample) - self.gamma_prior.log_prob(sample)
        
        
    def KLD(self, w1=1.0, w2=1.0):
        return self.kld_structure() *w1 + self.kld_normal() * w2

        
def nothing(x):
    return x
class ElboStructNet(nn.Module):
    def __init__(self, dims,  layer_num, temp_tensor):
        nn.Module.__init__(self)
        layers = []
        for l in range(layer_num):
            layers.append(MixedLayer(input_dim, dims, temp_tensor )) 
        layers.append(VarLayer(input_dim, 10,act=nothing)) 
            
        hyper_prior = 1.0
        self.model = nn.Sequential(*layers)
        self.hyper_gamma = t.distributions.HalfNormal(t.ones(1,device='cuda')*hyper_prior)
        
    def forward(self, x):
        return self.model(x)
        
    
    def hyper_ll(self):
        #https://github.com/stan-dev/stan/wiki/Prior-Choice-Recommendations
        k = 0
        for l in self.model:
            if isinstance(l, VarLayer):                                 
                    k+=self.hyper_gamma.log_prob(t.exp(l.h_sigma.view(-1))).sum()
                    k+=self.hyper_gamma.log_prob(t.exp(l.h_sigma_b.view(-1))).sum()
                
        return k
    

    def structure_params(self):
        result = []
        for l in self.model:
            if isinstance(l, MixedLayer):
                result.append(l.s)
                result.append(l.log_t)
                result.append(l.s_prior)
        return result
    
    
    def KLD(self, w1=1.0, w2 =1.0):
        k = 0
        for l in self.model:
            if isinstance(l, MixedLayer):
                k+=l.KLD( w1, w2)
            if isinstance(l, VarLayer):
                k+=l.KLD()
        return k
    
     

In [ ]:
def test_acc(model, loader, func = lambda x:x):
    tp = 0
    cases = 0
    model.eval()
    for x,y in loader: 
            x = func(x)
            x = x.cuda()
            y = y.cuda()
            out = model(x).argmax(1)
            tp+=(out==y).sum()
            cases+=len(y)
    model.train()
    return  tp.cpu().numpy()*1.0/cases

In [ ]:
import pickle
t.manual_seed(random_seed)
temp = t.ones(1).cuda()*0.2
for trial in range(trials):
    net = ElboStructNet(search_space, 4, temp)
    struct = net.structure_params()
    optimizer1 = optim.Adam([p for p in net.parameters() if p not in set(struct)], lr=0.001) 
    optimizer2 = optim.Adam(struct, lr=0.01) 

    loss_fn = nn.CrossEntropyLoss()    
    id=0
    for epoch in range(n_epochs):       
        for x,y in train_loader:
            id+=1        
            x = x.cuda()
            y = y.cuda()            
            optimizer1.zero_grad()
            optimizer2.zero_grad()
            loss = 0
            out_loss = 0
            kld =  t.zeros(1).cuda()             

            for _ in range(5):
                out = net(x)
                out_loss += loss_fn(out, y)*len(train_idx)*1.0/5       
            if epoch >= 0:
                kld+=net.KLD()
            hyper = -net.hyper_ll()

            loss = (out_loss+kld + hyper)       
            if id %10 == 0:
                print (net.model[0].gamma.probs.data, net.model[0].gamma.temperature)
                print ( out_loss.data, kld.data)
                
                print ('\n')

            loss.backward()
            clip_grad_value_(net.parameters(), 1.0)            
            optimizer1.step()  
            optimizer2.step()  

        acc = test_acc(net, valid_loader)            
        print ('Trial {0}. Epoch {1}. Acc: {2}'.format(trial, epoch, acc))
    # в pickle нельзя сохранить распределения
    # альтернатива: использовать встроенные средства сохранения в pytorch
    for m in net.model:
        try:
            del m.gamma
        except:
            pass 
        try:
            del m.lambda_t
        except:
            pass 
        try:
            del m.gamma_prior
        except:
            pass 
        try:
            del m.layer.eps_w
        except:
            pass 
        try:
            del m.layer.eps_b
        except:
            pass 
    with open( 'old_proposed{0}.pckl'.format(trial), 'wb') as out:
        pickle.dump(net, out)
    del net
    t.cuda.empty_cache()



In [ ]:
with open( 'full_proposed{0}.pckl'.format(0), 'rb') as inp:
        net = pickle.load(inp)

## Дообучение: Max Likelihood

In [ ]:
class AddLayer(nn.Module):
    def __init__(self, bias):
        nn.Module.__init__(self)
        self.bias = bias
    
    def forward(self, x):
        return x + self.bias
    
def elbonet_to_elbo(model):
    var_layers = []
    in_dim = input_dim
    for mixed_layer in model.model[:-1]:  
        
        if isinstance(mixed_layer, MixedLayer):
            best_layer_id = mixed_layer.s.argmax()
            if mixed_layer.dims[best_layer_id] == 1:
                continue
            print (best_layer_id)
            new_layer = VarLayer(in_dim, mixed_layer.dims[best_layer_id]).cuda()
            
            weights = t.zeros(mixed_layer.layer.mean.size()).cuda()+0.0001
            weights_b = t.zeros(mixed_layer.layer.mean_b.size()).cuda()+0.0001
            gamma = F.softmax(mixed_layer.s/mixed_layer.t)
            for i in range(len(mixed_layer.dims)):
                weights[:, :mixed_layer.dims[i]]+=gamma[i]
                weights_b[:mixed_layer.dims[i]] += gamma[i]
            weights = t.log(weights)
            weights_b = t.log(weights_b)
            
            new_layer.mean.data *= 0
            new_layer.mean.data += mixed_layer.layer.mean[:in_dim,:mixed_layer.dims[best_layer_id]]
            new_layer.log_sigma.data *= 0
            new_layer.log_sigma.data += mixed_layer.layer.log_sigma[:in_dim,:mixed_layer.dims[best_layer_id]]
            
            new_layer.mean_b.data *= 0
            new_layer.mean_b.data += mixed_layer.layer.mean_b[:mixed_layer.dims[best_layer_id]]
            new_layer.log_sigma_b.data *= 0
            new_layer.log_sigma_b.data += mixed_layer.layer.log_sigma_b[:mixed_layer.dims[best_layer_id]]
            
            new_layer.h_sigma.data *= 0
            new_layer.h_sigma.data += weights[:in_dim, :mixed_layer.dims[best_layer_id]]
            new_layer.h_sigma.data += mixed_layer.layer.h_sigma[:in_dim, :mixed_layer.dims[best_layer_id]]
            
            new_layer.h_sigma_b.data *= 0
            new_layer.h_sigma_b.data += weights_b[:mixed_layer.dims[best_layer_id]]
            new_layer.h_sigma_b.data += mixed_layer.layer.h_sigma_b[:mixed_layer.dims[best_layer_id]]
            
            
            var_layers.append(new_layer)
            var_layers.append(AddLayer(mixed_layer.noises[best_layer_id][:mixed_layer.dims[best_layer_id]]))
            
            in_dim = mixed_layer.dims[best_layer_id]
    
    sublayer = model.model[-1] 
    out_ = 10
    
    new_layer = VarLayer(in_dim, out_, act=lambda x:x).cuda()
    new_layer.mean.data *= 0
    new_layer.mean.data += sublayer.mean[:in_dim]
    new_layer.mean_b.data *= 0
    new_layer.mean_b.data += sublayer.mean_b[:in_dim]
    
    new_layer.h_sigma.data *= 0
    new_layer.h_sigma.data += sublayer.h_sigma[:in_dim]

    new_layer.h_sigma_b.data *= 0
    new_layer.h_sigma_b.data += sublayer.h_sigma_b[:in_dim]

            
    var_layers.append(new_layer)
    return nn.Sequential(*var_layers)

def elbonet_to_net(model):
    var_layers = []
    in_dim = input_dim
    for mixed_layer in model.model[:-1]:        
        if isinstance(mixed_layer, MixedLayer):
            best_layer_id = mixed_layer.s.argmax()
            if mixed_layer.dims[best_layer_id] == 1:
                continue
            print (best_layer_id)
            new_layer = nn.Linear(in_dim, mixed_layer.dims[best_layer_id]).cuda()
            
            
            new_layer.weight.data *= 0
            new_layer.weight.data += mixed_layer.layer.mean.transpose(1,0)[:mixed_layer.dims[best_layer_id], :in_dim]
            new_layer.bias.data *= 0
            new_layer.bias.data += mixed_layer.layer.mean_b[:mixed_layer.dims[best_layer_id]]
            var_layers.append(new_layer)
            var_layers.append(nn.Tanh())
            var_layers.append(AddLayer(mixed_layer.noises[best_layer_id][:mixed_layer.dims[best_layer_id]]))
            
            in_dim = mixed_layer.dims[best_layer_id]
    
    sublayer = model.model[-1] 
    out_ = 10
    
    new_layer =  nn.Linear(in_dim, out_).cuda()
    new_layer.weight.data *= 0
    new_layer.weight.data += sublayer.mean.transpose(1,0)[:, :in_dim]
    new_layer.bias.data *= 0
    new_layer.bias.data += sublayer.mean_b[:in_dim]
    
    
    
    var_layers.append(new_layer)
    return nn.Sequential(*var_layers)

#subnet = elbonet_to_net(net)
#subnet = elbonet_to_elbo(net)

In [ ]:
tuned_nets = []
for trial in range(trials):
    with open( 'old_proposed{0}.pckl'.format(trial), 'rb') as inp:
        net = pickle.load(inp)
    subnet = elbonet_to_net(net)
    optimizer1 = optim.Adam(subnet.parameters(), lr=0.001) # для параметров
    loss_fn = nn.CrossEntropyLoss()    
    id=0
    for epoch in range(fine_tune_epochs):        
        for x,y in train_loader:
            id+=1

            x = x.cuda()
            y = y.cuda()            
            optimizer1.zero_grad()        
            loss = 0
            out_loss = 0
            out = subnet(x)
            out_loss += loss_fn(out, y)*len(train_idx)*1.0        
            loss = (out_loss)       
            if id %100 == 0:            
                print (out_loss.data)
                print ('\n')

            loss.backward()
            clip_grad_value_(net.parameters(), 1.0)            
            optimizer1.step()  

        acc = test_acc(subnet, valid_loader)            
        print ('Trial {0}. Epoch {1}. Acc: {2}'.format(trial, epoch, acc))
    tuned_nets.append(subnet)


In [ ]:
with open('./old_proposed_tuned.pckl', 'wb') as out:
    pickle.dump(tuned_nets, out)

In [ ]:
stats = {}
pn = []
for subnet in tuned_nets:    
    num = 0
    for p in subnet.parameters():
    
        if len(p.size())==1:
            num+=p.size()[0]
        elif len(p.size())==0:
            num+=1
        else:
            num+=p.size()[1]*p.size()[0]
    pn.append(num)

stats['param number'] = pn
stats['param number']


In [ ]:
def get_superposition_number(): 
    sn = []
    for subnet in tuned_nets:
        cnt = 0
        for submodel in subnet:
            print (submodel)
            if len(list(submodel.parameters()))>0:
                cnt+=1
        sn.append(cnt)
        
    return sn
stats['superposition number'] = get_superposition_number()
stats['superposition number']

In [ ]:
t.manual_seed(random_seed)
X = []
Y = []
Y_std = []
accs = []
for noise in np.linspace(0, 1.0, 10):
    X.append(noise)
    acc = []
    for subnet in tuned_nets:
                     
        acc += [test_acc(subnet, test_loader, func = lambda x: x+t.randn(x.size())*noise)] 
    print (acc)
    Y.append(np.mean(acc))
    Y_std.append(np.std(acc))
    accs.append(acc)

    

In [ ]:
stats['noise'] = [X,Y,Y_std, accs]

In [ ]:
t.manual_seed(random_seed)
X = []
Y = []
accs = []
Y_std = []
for noise in np.linspace(0, 0.1, 10):
    X.append(noise)
    acc = []
    for subnet in tuned_nets:
        m = subnet
        m.eval()
        old_params = []
        for p in m.parameters():
            old_params.append(p.data*1.0)

        tp = 0        
        for x,y in test_loader:

            for p, o in zip(m.parameters(), old_params):                
                n = t.randn(p.data.shape)*noise
                n = n.cuda()                    
                p.data = o + n
            x = x.cuda()
            y = y.cuda()
            out = m(x).argmax(1)
            tp+=(out==y).sum()
            for p, o in zip(m.parameters(), old_params):                
                p.data = o
        acc.append(tp.cpu().numpy()*1.0/len(test_data))
    print (acc)
    accs.append(acc)
    Y.append(np.mean(acc))
    Y_std.append(np.std(acc))
stats['params'] = [X,Y,Y_std, accs]

In [ ]:
with open('./old_proposed_ml_stats.pckl', 'wb') as out:
    pickle.dump(stats, out)

## Дообучение: ELBO

In [ ]:
import pickle
tuned_nets = []
for trial in range(trials):
    with open( 'old_proposed{0}.pckl'.format(trial), 'rb') as inp:
        net = pickle.load(inp)
    subnet = elbonet_to_elbo(net)
    optimizer1 = optim.Adam(subnet.parameters(), lr=0.001) # для параметров
    loss_fn = nn.CrossEntropyLoss()    
    id=0
    for epoch in range(fine_tune_epochs):        
        for x,y in train_loader:
            id+=1

            x = x.cuda()
            y = y.cuda()            
            optimizer1.zero_grad()        
            loss = 0
            out_loss = 0
            out = subnet(x)
            for _ in range(5):
                out_loss += loss_fn(out, y)*len(train_idx)*1.0/5
            k = 0
            for m in subnet:
                if isinstance(m, VarLayer):
                    k+=m.KLD()
            loss = (out_loss+k)       
            if id %100 == 0:            
                print (out_loss.data, k.data)
                print ('\n')

            loss.backward()
            clip_grad_value_(net.parameters(), 1.0)            
            optimizer1.step()  

        acc = test_acc(subnet, valid_loader)            
        print ('Trial {0}. Epoch {1}. Acc: {2}'.format(trial, epoch, acc))
    tuned_nets.append(subnet)


In [ ]:
for n in tuned_nets:
   n[-1].act = nothing

In [ ]:
with open('./old_proposed_tuned_var.pckl', 'wb') as out:
    pickle.dump(tuned_nets, out)

In [ ]:
with open('./old_proposed_tuned_var.pckl', 'rb') as inp:
    tuned_nets = pickle.load(inp)

In [ ]:
stats = {}

In [ ]:
t.manual_seed(random_seed)
X = []
Y = []
Y_std = []
accs = []
for noise in np.linspace(0, 1.0, 10):
    X.append(noise)
    acc = []
    for subnet in tuned_nets:
                     
        acc += [test_acc(subnet, test_loader, func = lambda x: x+t.randn(x.size())*noise)] 
    print (acc)
    Y.append(np.mean(acc))
    Y_std.append(np.std(acc))
    accs.append(acc)
stats['noise'] =[X,Y,Y_std, accs]

In [ ]:
t.manual_seed(random_seed)
X = []
Y = []
accs = []
Y_std = []
for noise in np.linspace(0, 0.1, 10):
    X.append(noise)
    acc = []
    for subnet in tuned_nets:
        subnet.eval()
        m = subnet

        old_params = []
        for p in m.parameters():
            old_params.append(p.data*1.0)

        tp = 0        
        for x,y in test_loader:

            for p, o in zip(m.parameters(), old_params):                
                n = t.randn(p.data.shape)*noise
                n = n.cuda()                    
                p.data = o + n
            
            x = x.cuda()
            y = y.cuda()
            out = m(x).argmax(1)
            tp+=(out==y).sum()
            for p, o in zip(m.parameters(), old_params):                
                p.data = o
        acc.append(tp.cpu().numpy()*1.0/len(test_data))
    print (acc)
    accs.append(acc)
    Y.append(np.mean(acc))
    Y_std.append(np.std(acc))
stats['params'] = [X,Y,Y_std, accs]

In [ ]:
with open('./old_proposed_var_stats.pckl', 'wb') as out:
    pickle.dump(stats, out)